<a href="https://colab.research.google.com/github/edcalderin/llm-ml-experiments/blob/master/these_are_the_3_langchain_functions_i_used_to_improve_my_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# These Are the 3 Langchain Functions I Used to Improve My RAG

https://ai.gopubby.com/these-are-the-3-langchain-functions-i-used-to-improve-my-rag-00413ccb7094

## Multi Query Retriever

In [1]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [6]:
!pip install -q langchain-openai langchain-community chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.3 MB/s eta 

In [26]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma
from langchain import hub
from langchain.retrievers import MultiQueryRetriever
from bs4 import BeautifulSoup, SoupStrainer

In [53]:
url: str = "https://es.wikipedia.org/wiki/Accidente_del_transbordador_espacial_Challenger"

loader = WebBaseLoader(
    url,
    bs_kwargs={"parse_only": SoupStrainer(class_="mw-body-content")}
)

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=10)
splits = text_splitter.split_documents(documents)

print(len(splits), "splits")
print(splits[:6])
vector_store = Chroma.from_documents(splits, OpenAIEmbeddings(), persist_directory="multi_query")

537 splits
[Document(metadata={'source': 'https://es.wikipedia.org/wiki/Accidente_del_transbordador_espacial_Challenger'}, page_content='Accidente del transbordador espacial Challenger\nMontaje fotográfico del desastre del transbordador espacial Challenger.Suceso'), Document(metadata={'source': 'https://es.wikipedia.org/wiki/Accidente_del_transbordador_espacial_Challenger'}, page_content='Desintegración del transbordador y posterior caída de restos al océano AtlánticoFecha\n28 de enero de 1986 (39 años,\xa06 meses\xa0y\xa08 días)Hora\n11:39:13 EST (16:39:13 UTC)Causa'), Document(metadata={'source': 'https://es.wikipedia.org/wiki/Accidente_del_transbordador_espacial_Challenger'}, page_content='Falla en junta tórica del cohete acelerador sólido (SRB) derecho del trasbordadorLugar\nOcéano Atlántico, frente a la costa de FloridaResultado'), Document(metadata={'source': 'https://es.wikipedia.org/wiki/Accidente_del_transbordador_espacial_Challenger'}, page_content='Puesta en tierra de la flo

In [74]:
llm = ChatOpenAI(model="gpt-4o")

retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 3})

multi_query_retriever = MultiQueryRetriever.from_llm(retriever, llm)

prompt = hub.pull("rlm/rag-prompt")

chain = (
    {
        "question": RunnablePassthrough(),
        "context": multi_query_retriever
    }
    | prompt
    | llm
    | StrOutputParser()
)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [75]:
chain.invoke("En que año exploto el transbordador challenger?")

'El transbordador Challenger explotó en el año 1986.'

In [76]:
chain.invoke("Cuantas personas iban dentro del transbordador challenger?")

'En el Challenger iban siete personas a bordo.'

In [78]:
chain.invoke("Cuanto tiempo paso luego de haber sido lanzado el transbordador?")

'No tengo suficiente información en el contexto proporcionado para determinar cuánto tiempo pasó después de que el transbordador fue lanzado.'

## Long Context Recorder

In [79]:
from langchain_community.document_transformers import LongContextReorder

In [80]:
url: str = "https://es.wikipedia.org/wiki/Accidente_del_transbordador_espacial_Challenger"

loader = WebBaseLoader(
    url,
    bs_kwargs={"parse_only": SoupStrainer(class_="mw-body-content")}
)

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=10)

splits = text_splitter.split_documents(documents)

vector_store = Chroma.from_documents(splits, OpenAIEmbeddings(), persist_directory="long_context_recorder_data")

In [94]:
llm = ChatOpenAI(model="gpt-4o")

retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 20})

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs: list)->list:
    context_recorder = LongContextReorder()
    return context_recorder.transform_documents(docs)

chain = (
    {
        "question": RunnablePassthrough(),
        "context": retriever | format_docs
    }
    | prompt
    | llm
    | StrOutputParser()
)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [95]:
chain.invoke("En que año exploto el transbordador challenger?")

'El transbordador espacial Challenger explotó el 28 de enero de 1986.'

In [98]:
chain.invoke("Cuantas personas iban dentro del transbordador challenger?")

'Iban siete personas a bordo del transbordador Challenger.'

In [97]:
chain.invoke("Cuanto tiempo paso luego de haber sido lanzado el transbordador?")

'El transbordador espacial Challenger se desintegró 73 segundos después de haber sido lanzado el 28 de enero de 1986.'

## Contextual Compression

In [99]:
url: str = "https://es.wikipedia.org/wiki/Accidente_del_transbordador_espacial_Challenger"

loader = WebBaseLoader(
    url,
    bs_kwargs={"parse_only": SoupStrainer(class_="mw-body-content")}
)

documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=10)

splits = text_splitter.split_documents(documents)

vector_store = Chroma.from_documents(splits, OpenAIEmbeddings(), persist_directory="contextual_compression_data")

In [136]:
from langchain.retrievers.document_compressors import (
    DocumentCompressorPipeline, EmbeddingsFilter
)
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_transformers import (
    EmbeddingsRedundantFilter
)
from langchain.retrievers import ContextualCompressionRetriever

splitter = CharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=0, # Reduced overlap to avoid large chunks
    separator=". "
)

embedding = OpenAIEmbeddings()

redundant_filter = EmbeddingsRedundantFilter(embeddings=embedding)

relevant_filter = EmbeddingsFilter(
    embeddings=embedding,
    similarity_threshold=0.8 # Increased threshold to filter more documents
)

pipeline_compressor = DocumentCompressorPipeline(
    transformers=[splitter, redundant_filter, relevant_filter]
)

retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 5}
)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor, base_retriever=retriever
)

In [137]:
llm = ChatOpenAI(model="gpt-4o")

prompt = hub.pull("rlm/rag-prompt")

chain = (
    {
        "question": RunnablePassthrough(),
        "context": compression_retriever
    }
    | prompt
    | llm
    | StrOutputParser()
)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [138]:
chain.invoke("En que año exploto el transbordador challenger?")

'El transbordador espacial Challenger explotó el 28 de enero de 1986.'

In [139]:
chain.invoke("Cuantas personas iban dentro del transbordador challenger?")

'El transbordador Challenger llevaba a bordo siete personas.'

In [140]:
chain.invoke("Cuanto tiempo paso luego de haber sido lanzado el transbordador?")

'El transbordador espacial Challenger fue lanzado el 28 de enero de 1986, y después de una pausa de 32 meses en el programa de transbordadores, la siguiente misión, la STS-26, se lanzó el 29 de septiembre de 1988.'